In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from astropy.table import Table

d = Table.read('/content/drive/MyDrive/catalogs.hdf5')

In [3]:
import numpy as np

x = np.array(d['image'][:16000]).reshape((-1, 101, 101, 1))
y = np.array(d['is_lens'][:16000]).reshape((-1, 1))

x_test = np.array(d['image'][16000:]).reshape((-1, 101, 101, 1))
y_test = np.array(d['is_lens'][16000:]).reshape((-1, 1))

In [4]:
import os
from glob import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

#datagenerator 생성
datagenerator_1 = ImageDataGenerator(
                                    rotation_range = 120,
                                    vertical_flip = True,
                                    horizontal_flip = True)

datagenerator_2 = ImageDataGenerator(
                                    rotation_range = 90,
                                    vertical_flip = False,
                                    horizontal_flip = True)

def generating1(x,y):
  x0 = np.array(x)
  y0 = np.array(y)
  p = datagenerator_1.flow(x0,y0,batch_size=len(x)-800)
  print(len(x))

  return p

def generating2(x,y):
  x0 = np.array(x)
  y0 = np.array(y)
  p = datagenerator_2.flow(x0,y0,batch_size=len(x)-3000)
  print(len(x))

  return p

def data_shuffle(x,y):
  xt = []
  yt = []
  entire = list(zip(x,y))
  random.shuffle(entire)

  for i in range(len(x)):
    xt.append(entire[i][0])
    yt.append(entire[i][1])

  x = np.asarray(xt).astype(np.float32)
  y = np.asarray(yt).astype(np.float32)

  return x, y

x0 = []
x1 = []

for i in range(len(x)):
  if y[i]==0:
    x0.append(x[i])
  elif y[i]==1:
    x1.append(x[i])

y0 = [0]*len(x0)
y1 = [1]*len(x1)

x = list(x)
y = list(y)

im,lb = generating1(x0,y0).next()
im1,lb1 = generating2(x1,y1).next()

for i in range(len(im)):
  x0.append(im[i])
  y0.append(lb[i])


for i in range(len(im1)):
  x1.append(im1[i])
  y1.append(lb1[i])


#x1 = x1[:len(x0)]
#y1 = [1]*len(x0)

x = x0+x1
y = y0+y1
x,y = data_shuffle(x,y)

print(x.shape)


'''
im, lb = generating(x0,y)

p1 = datagenerator_1.flow(x,y,batch_size=16000)
p2 = datagenerator_2.flow(x,y,batch_size=6000)
x = list(x)
y = list(y)

xa,ya = p1.next()
xaa,yaa = p2.next()
xa = np.array(xa)
ya = np.array(ya)
xaa = np.array(xaa)
yaa = np.array(yaa)

for i in range(len(xa)):
  x.append(xa[i])
  y.append(ya[i])

for i in range(len(xaa)):
  x.append(xaa[i])
  y.append(yaa[i])

x = np.array(x)
y = np.array(y)

print(x.shape)
print(y.shape)
'''

4834
11166
(28200, 101, 101, 1)


'\nim, lb = generating(x0,y)\n\np1 = datagenerator_1.flow(x,y,batch_size=16000)\np2 = datagenerator_2.flow(x,y,batch_size=6000)\nx = list(x)\ny = list(y)\n\nxa,ya = p1.next()\nxaa,yaa = p2.next()\nxa = np.array(xa)\nya = np.array(ya)\nxaa = np.array(xaa)\nyaa = np.array(yaa)\n\nfor i in range(len(xa)):\n  x.append(xa[i])\n  y.append(ya[i])\n\nfor i in range(len(xaa)):\n  x.append(xaa[i])\n  y.append(yaa[i])\n\nx = np.array(x)\ny = np.array(y)\n\nprint(x.shape)\nprint(y.shape)\n'

In [5]:
vmin = -1e-9
vmax = 1e-9
scale = 100

mask = np.where(x == 100)
mask_test = np.where(x_test == 100)

x[mask] = 0
x_test[mask_test] = 0

x = np.clip(x, vmin, vmax)/vmax * scale
x_test = np.clip(x_test, vmin, vmax)/vmax * scale

x[mask] = 0
x_test[mask_test] = 0

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

def resnet_block(input_layer, num_cnn=3, channel=64, block_num=1):
  x = input_layer
  for cnn_num in range(num_cnn):
    identity = x
    x = layers.Conv2D(filters=channel, kernel_size=(1,1), activation='elu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=channel, kernel_size=(3,3), activation='elu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=channel, kernel_size=(1,1), activation='elu', kernel_initializer='he_normal', padding='same')(x)
    #x = layers.BatchNormalization()(x)
  
  x = layers.Add()([x,identity])
  x = layers.MaxPooling2D(pool_size=(2,2))(x)

  return x

def build_resnet(input_shape=(101,101,1), num_cnn_list=[3,4,6,3], channel_list=[64,128,256,512], num_classes=2):
  assert len(num_cnn_list) == len(channel_list)

  input_layer = layers.Input(shape=input_shape)
  output = input_layer
  output = layers.Conv2D(filters=16, kernel_size=(7,7), kernel_initializer='he_normal', padding='same')(output)
  output = layers.BatchNormalization()(output)

  #output = layers.MaxPooling2D(pool_size=(2,2))(output)

  for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
    output = resnet_block(output, num_cnn=num_cnn, channel=channel, block_num=i)

  output = layers.AveragePooling2D(padding='same')(output)
  output = layers.Flatten()(output)
  #output = layers.Dropout(0.35)(output)
  output = layers.Dense(1, activation='sigmoid')(output)

  model = tf.keras.Model(inputs=input_layer, outputs=output)
  return model

In [7]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import tensorflow_addons as tfa

resnet = build_resnet(num_cnn_list=[3,3,3,3,3], channel_list=[16,32,64,128,256])

first_decay_steps = 1000
initial_learning_rate = 0.01

wd = lambda: 1e-4
optimizer = tfa.optimizers.AdamW(weight_decay=wd)

resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 101, 101, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 101, 101, 16  800         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 101, 101, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
stop = tf.keras.callbacks.EarlyStopping()

period = 5
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/CRA_vacation_project/parameters/model_tcmu_nod_{epoch}.ckpt', save_weights_only=False, verbose=1, period=period)

resnet.fit(x, y, epochs=60, validation_split=0.2, callbacks=[cp_callback])

Epoch 1/60
339/705 [=============>................] - ETA: 28s - loss: 0.7715 - accuracy: 0.6721

In [ ]:
resnet.evaluate(x_test, y_test, batch_size=10)